<a href="https://colab.research.google.com/github/hmsquare/DL2024final/blob/VQA-competition/beit3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## installs

In [ ]:
#!git clone https://github.com/microsoft/unilm.git
# %cd /content/drive/MyDrive/03_AY2425/04_5_DL/_final/
# !git clone https://github.com/hmsquare/unilm.git
%cd /content/drive/MyDrive/03_AY2425/04_5_DL/_final/unilm/beit3
# !git pull
!pip install -r requirements.txt

/content/drive/MyDrive/03_AY2425/04_5_DL/_final/unilm/beit3
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 8.76 KiB | 114.00 KiB/s, done.
From https://github.com/hmsquare/unilm
   3421a97..9d724a3  master     -> origin/master
Updating 3421a97..9d724a3
^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.2/398.2 kB 42.7 MB/s eta 0:00:00
  

## imports

In [ ]:
import numpy as np
import pandas as pd
import operator
import os
import string
import re
import random
import sys
import platform
import json
import shutil

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from PIL import Image

from transformers import XLMRobertaTokenizer

from tqdm import tqdm

#from glossary import normalize_word
from collections import Counter, defaultdict
import glob

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def normalize_word_nltk(text):
    lemmatizer = WordNetLemmatizer()
    words = text.lower().strip().split()  # Split the string into words
    normalized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word

    return ' '.join(normalized_words)  # Join the words back into a single string

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
CFG = {
    # 'EPOCHS': 30,
    # 'LEARNING_RATE': 2e-5,
    # 'BATCH_SIZE': 32,
    'SEED': 42,
    # 'MAX_LEN': 16,
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# data transformation

## for custom

In [ ]:
tf = open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/train.json')
train_json = json.load(tf)

items_train = []
images_train = list(train_json['image'].items())
questions_train = list(train_json['question'].items())
answers_train = list(train_json['answers'].items())

for i, img in images_train:
  item = {}
  item['id'] = int(i)
  item['image'] = img
  item['question'] = questions_train[int(i)][1]
  item['answer'] = answers_train[int(i)][-1]

  items_train.append(item)


train_items, val_items = train_test_split(items_train, test_size=0.15, random_state=CFG['SEED'])

with open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beit_json/train.json', "w") as json_file:
  json.dump(train_items, json_file)

with open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beit_json/val.json', "w") as json_file:
  json.dump(val_items, json_file)

In [ ]:
vf = open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/valid.json')
valid_json = json.load(vf)

items_valid = []
images_valid = list(valid_json['image'].items())
questions_valid = list(valid_json['question'].items())


for i, img in images_valid:
  item = {}
  item['id'] = int(i)
  item['image'] = img
  item['question'] = questions_valid[int(i)][1]

  items_valid.append(item)


with open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beit_json/test.json', "w") as json_file:
  json.dump(items_valid, json_file)

## for vqav2

In [ ]:
btft = open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beit_json/train.json')
btft_data = json.load(btft)
newdict_t, trainimgs, trainqs, trainans = {}, [], [],[]
for data in btft_data:
  imgdict = {}
  imgdict['image_id'] = data['image'].split('_')[-1][:5]
  imgdict['image_path'] = data['image']
  trainimgs.append(imgdict)

  qdict = {}
  qdict['image_id'] = data['image'].split('_')[-1][:5]
  qdict['question'] = data['question']
  qdict['question_id'] = int(data['image'].split('_')[-1][:5])*1000
  trainqs.append(qdict)

  adict = {}
  adict['image_id'] = data['image'].split('_')[-1][:5]
  adict['question_id'] = int(data['image'].split('_')[-1][:5])*1000
  adict['multiple_choice_answer'] = Counter([ansdict['answer'] for ansdict in data['answer']]).most_common()[0][0]
  adict['answer'] = data['answer']
  trainans.append(adict)

newdict_t['images'] = trainimgs
newdict_t['questions'] = trainqs
newdict_t['annotations'] = trainans

with open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beitvqa_json/train.json', "w") as json_file:
  json.dump(newdict_t, json_file)

In [ ]:
newdict_t

{'images': [{'image_id': '12213', 'image_path': 'train_12213.jpg'},
  {'image_id': '11782', 'image_path': 'train_11782.jpg'},
  {'image_id': '06455', 'image_path': 'train_06455.jpg'},
  {'image_id': '02066', 'image_path': 'train_02066.jpg'},
  {'image_id': '13405', 'image_path': 'train_13405.jpg'},
  {'image_id': '03588', 'image_path': 'train_03588.jpg'},
  {'image_id': '02271', 'image_path': 'train_02271.jpg'},
  {'image_id': '17496', 'image_path': 'train_17496.jpg'},
  {'image_id': '19289', 'image_path': 'train_19289.jpg'},
  {'image_id': '09226', 'image_path': 'train_09226.jpg'},
  {'image_id': '08886', 'image_path': 'train_08886.jpg'},
  {'image_id': '11909', 'image_path': 'train_11909.jpg'},
  {'image_id': '05163', 'image_path': 'train_05163.jpg'},
  {'image_id': '18132', 'image_path': 'train_18132.jpg'},
  {'image_id': '13071', 'image_path': 'train_13071.jpg'},
  {'image_id': '14093', 'image_path': 'train_14093.jpg'},
  {'image_id': '09482', 'image_path': 'train_09482.jpg'},
  {'

In [ ]:
btfv = open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beit_json/val.json')
btfv_data = json.load(btfv)
newdict_v, valqs, valans, valimgs = {}, [], [], []
for data in btfv_data:
  imgdict = {}
  imgdict['image_id'] = data['image'].split('_')[-1][:5]
  imgdict['image_path'] = data['image']
  valimgs.append(imgdict)

  qdict = {}
  qdict['image_id'] = data['image'].split('_')[-1][:5]
  qdict['question'] = data['question']
  qdict['question_id'] = int(data['image'].split('_')[-1][:5])*1000
  valqs.append(qdict)

  adict = {}
  adict['image_id'] = data['image'].split('_')[-1][:5]
  adict['question_id'] = int(data['image'].split('_')[-1][:5])*1000
  adict['multiple_choice_answer'] = Counter([ansdict['answer'] for ansdict in data['answer']]).most_common()[0][0]
  adict['answer'] = data['answer']
  valans.append(adict)

newdict_t['images'] = trainimgs
newdict_v['questions'] = valqs
newdict_v['annotations'] = valans

with open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beitvqa_json/val.json', "w") as json_file:
  json.dump(newdict_v, json_file)

In [ ]:
# # = = = debug area = = =
# ama = [d['multiple_choice_answer'] for d in trainans + valans]
# all_major_answers = [normalize_word_nltk(word) for word in ama]
# counter = {k: v for k, v in Counter(all_major_answers).items()} # if v >= 9} # for choosing the major ans
# ans2label = {k: i for i, k in enumerate(counter.keys())}
# label2ans = list(counter.keys())

# normalize_word_nltk('csi cant stand idiots')

In [ ]:
btfs = open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beit_json/test.json')
btfs_data = json.load(btfs)
newdict_s, testims, testqs = {}, [], []
for data in btfs_data:
  imgdict = {}
  imgdict['image_id'] = data['image'].split('_')[-1][:5]
  imgdict['image_path'] = data['image']
  testims.append(imgdict)

  qdict = {}
  qdict['image_id'] = data['image'].split('_')[-1][:5]
  qdict['question'] = data['question']
  qdict['question_id'] = int(data['image'].split('_')[-1][:5])*1000
  testqs.append(qdict)

newdict_s['questions'] = testqs
newdict_s['images'] = testims

with open('/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beitvqa_json/test.json', "w") as json_file:
  json.dump(newdict_s, json_file)

In [ ]:
newdict_s

{'questions': [{'image_id': '00000',
   'question': 'Was I able to clear either of the mirrors of the OK button?',
   'question_id': 0},
  {'image_id': '00001',
   'question': 'What page number is this above? Thank you.',
   'question_id': 1000},
  {'image_id': '00002',
   'question': 'Please tell me what is in this box.',
   'question_id': 2000},
  {'image_id': '00003',
   'question': 'Are the lights on in this room?',
   'question_id': 3000},
  {'image_id': '00004',
   'question': 'What color is this? Please, thank you.',
   'question_id': 4000},
  {'image_id': '00005',
   'question': 'Any writing on this?',
   'question_id': 5000},
  {'image_id': '00006',
   'question': 'What kind of candy bar is this?',
   'question_id': 6000},
  {'image_id': '00007',
   'question': 'What color is this?',
   'question_id': 7000},
  {'image_id': '00008', 'question': 'What are these?', 'question_id': 8000},
  {'image_id': '00009',
   'question': 'What, uh, color is this? Is this dark green or blue, a

# pytorch dataset

## required classes

### BaseDataset

In [ ]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(
        self, data_path, split, transform,
        tokenizer, num_max_bpe_tokens, task=None,
    ):
        index_files = self.get_index_files(split, task=task)
        self.tokenizer = tokenizer
        self.num_max_bpe_tokens = num_max_bpe_tokens
        self.data_path = data_path
        items = []
        self.index_files = index_files

        offset = 0
        for _index_file in index_files:
            index_file = os.path.join(data_path, _index_file)
            with open(index_file, mode="r", encoding="utf-8") as reader:
                for line in reader:
                    data = json.loads(line)
                    items.append(data)
                print("Load %d image-text pairs from %s. " % (len(items) - offset, index_file))
                offset = len(items)
        self.items = items
        self.bos_token_id = tokenizer.bos_token_id
        self.eos_token_id = tokenizer.eos_token_id
        self.pad_token_id = tokenizer.pad_token_id
        self.loader = default_loader
        self.transform = transform
        self.split = split

    @staticmethod
    def get_index_files(split):
        raise NotImplementedError()

    def _get_image(self, image_path: str):
        image_path = os.path.join(self.data_path, image_path)
        image = self.loader(image_path)
        return self.transform(image)

    def _get_text_segment(self, text_segment, max_len=None):
        if isinstance(text_segment, str):
            tokens = self.tokenizer.tokenize(text_segment)
        else:
            tokens = text_segment[:]
        if len(tokens) == 0:
            raise RuntimeError("The text segment should contains at least one tokens!")
        if max_len is None:
            max_len = self.num_max_bpe_tokens

        if len(tokens) > max_len - 2:
            tokens = tokens[:max_len - 2]

        tokens = [self.bos_token_id] + tokens[:] + [self.eos_token_id]
        num_tokens = len(tokens)
        padding_mask = [0] * num_tokens + [1] * (max_len - num_tokens)
        return tokens + [self.pad_token_id] * (max_len - num_tokens), padding_mask, num_tokens

    def _get_image_text_example(self, index: int, data: dict):
        item = self.items[index]
        img_path = item["image_path"]
        img = self._get_image(img_path)
        data["image"] = img

        text_segment = item["text_segment"]
        language_tokens, padding_mask, _ = self._get_text_segment(text_segment)
        data["language_tokens"] = language_tokens
        data["padding_mask"] = padding_mask

    def __getitem__(self, index: int):
        data = dict()
        self._get_image_text_example(index, data)
        return data

    def __len__(self) -> int:
        return len(self.items)

    def __repr__(self) -> str:
        head = "Dataset " + self.__class__.__name__
        body = '{' + "\n  Number of items: %s," % self.__len__()
        body += "\n  data root = %s," % self.data_path
        body += "\n  split = %s," % self.split
        body += "\n  dataset index files = %s" % str(self.index_files)
        body += "\n  num max bpe tokens = %s" % self.num_max_bpe_tokens
        body += "\n  transforms = ["
        for t in self.transform.transforms:
            body += "\n    %s" % str(t)
        body += "\n  ]"
        body += "\n}"

        return head + body


def _write_data_into_jsonl(items, jsonl_file):
    with open(jsonl_file, mode="w", encoding="utf-8") as writer:
        for data in items:
            writer.write(json.dumps(data, indent=None))
            writer.write('\n')
    print("Write %s with %d items !" % (jsonl_file, len(items)))


def _make_retrieval_coco_karpathy_dataset_index(
        data_path,
        tokenizer,
        split=("train", "restval"),
        split_name="train",
):
    coco_karpathy_split_json_file = os.path.join(data_path, "dataset_coco.json")
    items = []
    image_counter = set()
    print("read %s" % coco_karpathy_split_json_file)
    with open(coco_karpathy_split_json_file, mode="r", encoding="utf-8") as reader:
        data = json.loads(reader.read())
        for item in data["images"]:
            if item["split"] in split:
                image_path = os.path.join(item["filepath"], item["filename"])
                for sent in item["sentences"]:
                    tokens = tokenizer.tokenize(sent["raw"])
                    token_ids = tokenizer.convert_tokens_to_ids(tokens)
                    items.append({
                            "image_path": image_path,
                            "text_segment": token_ids,
                            "image_id": len(image_counter),
                    })
                if image_path not in image_counter:
                    image_counter.add(image_path)
    print("Find %d images and %d image-text pairs for karpathy dataset %s split !" % \
        (len(image_counter), len(items), split_name))
    index_file = os.path.join(data_path, "coco_retrieval.%s.jsonl" % split_name)
    _write_data_into_jsonl(items, index_file)
    pass


def _make_captioning_coco_karpathy_dataset_index(
        data_path,
        tokenizer,
        split=("train", "restval"),
        split_name="train",
):
    coco_karpathy_split_json_file = os.path.join(data_path, "dataset_coco.json")
    items = []
    image_counter = set()
    print("read %s" % coco_karpathy_split_json_file)
    with open(coco_karpathy_split_json_file, mode="r", encoding="utf-8") as reader:
        data = json.loads(reader.read())
        for item in data["images"]:
            if item["split"] in split:
                image_path = os.path.join(item["filepath"], item["filename"])
                if item["split"] in ["train", "restval"]:
                    for sent in item["sentences"]:
                        tokens = tokenizer.tokenize(sent["raw"])
                        token_ids = tokenizer.convert_tokens_to_ids(tokens)
                        items.append({
                                "image_path": image_path,
                                "text_segment": token_ids,
                                "image_id": item["cocoid"],
                        })
                else:
                    items.append({
                                "image_path": image_path,
                                "text_segment": None,
                                "image_id": item["cocoid"],
                    })
                if image_path not in image_counter:
                    image_counter.add(image_path)
    print("Find %d images and %d image-text pairs for karpathy dataset %s split !" % \
        (len(image_counter), len(items), split_name))
    index_file = os.path.join(data_path, "coco_captioning.%s.jsonl" % split_name)
    _write_data_into_jsonl(items, index_file)
    pass


def _make_nocaps_dataset_index(
        data_path,
        split="val",
):
    if split == "val":
        json_file = "nocaps_val_4500_captions.json"
    elif split == "test":
        json_file = "nocaps_test_image_info.json"
    nocaps_split_json_file = os.path.join(data_path, json_file)
    items = []
    image_counter = set()
    print("read %s" % nocaps_split_json_file)
    with open(nocaps_split_json_file, mode="r", encoding="utf-8") as reader:
        data = json.loads(reader.read())
        for item in data["images"]:
            image_path = os.path.join(split, item["file_name"])
            items.append({
                "image_path": image_path,
                "text_segment": None,
                "image_id": item["id"],
            })

            if image_path not in image_counter:
                image_counter.add(image_path)

    print("Find %d images and %d image-text pairs for nocaps dataset %s split !" % \
        (len(image_counter), len(items), split))
    index_file = os.path.join(data_path, "nocaps.%s.jsonl" % split)
    _write_data_into_jsonl(items, index_file)

### VQAv2

In [ ]:
class VQAv2Dataset(BaseDataset):
    def __init__(self, data_path, **kwargs):
        super().__init__(data_path=data_path, **kwargs)
        ans2label_file = os.path.join(data_path, "answer2label.txt")
        ans2label = {}
        label2ans = []
        with open(ans2label_file, mode="r", encoding="utf-8") as reader:
            for i, line in enumerate(reader):
                data = json.loads(line)
                ans = data["answer"]
                label = data["label"]
                label = int(label)
                assert label == i
                ans2label[ans] = i
                label2ans.append(ans)

        self.ans2label = ans2label
        self.label2ans = label2ans

    @staticmethod
    def get_index_files(split, task=None):
        if split == "train":
            return ("vqa.train.jsonl", "vqa.trainable_val.jsonl")
        elif split == "val":
            return ("vqa.rest_val.jsonl", )
        elif split == "test":
            return ("vqa.test.jsonl", )
        elif split == "test-dev":
            return ("vqa.test-dev.jsonl", )
        else:
            raise RuntimeError("split %s is not found!" % split)

    def __getitem__(self, index: int):
        data = super().__getitem__(index)
        if "labels" in self.items[index] and len(self.items[index]["labels"]) > 0:
            labels = [0.] * len(self.label2ans)
            for l, s in zip(self.items[index]["labels"], self.items[index]["scores"]):
                labels[l] = s
            data["labels"] = torch.FloatTensor(labels)
        else:
            data["qid"] = self.items[index]["qid"]
        return data

    @staticmethod
    def get_score(occurences):
        if occurences == 0:
            return 0.0
        elif occurences == 1:
            return 0.3
        elif occurences == 2:
            return 0.6
        elif occurences == 3:
            return 0.9
        else:
            return 1.0

    @classmethod
    def make_dataset_index(cls, data_path, tokenizer, annotation_data_path):
      # = = = LOAD JSON DATA = = =
        with open(os.path.join(annotation_data_path, "train.json"), "r") as fp:
            train_data = json.load(fp)
            questions_train = train_data['questions']
            annotations_train = train_data['annotations']
        with open(os.path.join(annotation_data_path, "val.json"), "r") as fp:
            val_data = json.load(fp)
            questions_val = val_data['questions']
            annotations_val = val_data['annotations']
        with open(os.path.join(annotation_data_path, "test.json"), "r") as fp:
            test_data = json.load(fp)
            questions_test = test_data['questions']

        annotations = dict()

        # = = = PROCESS QUESTIONS = = =
        for split, questions in zip(
            ["train", "val", "test"],
            [questions_train, questions_val, questions_test],
        ):
            _annot = defaultdict(dict)
            for q in questions:
                question_text = q['question']
                tokens = tokenizer.tokenize(question_text)
                token_ids = tokenizer.convert_tokens_to_ids(tokens)

                assert q["question_id"] not in _annot[int(q["image_id"])]
                _annot[int(q["image_id"])][q["question_id"]] = {
                    "question": question_text,
                    "token_ids": token_ids,
                }

            annotations[split] = _annot

        # ====== TROUBLESHOOT POINT ======
        print("Loaded annotations after question processing:")
        for split, qlist in zip(["train", "val", "test"],[questions_train, questions_val, questions_test]):
            print(f"{split} annotations: {len(annotations[split])} out of {len(qlist)}")


        # = = = PROCESS ANSWERS = = =

        all_major_answers = list()

        for split, annots in zip(
            ["train", "val"], [annotations_train, annotations_val],
        ):
            # _annot = annotations[split]
            for q in annots:
                all_major_answers.append(q["multiple_choice_answer"])

        print('len all_major_answers:', len(all_major_answers))

        all_major_answers = [normalize_word_nltk(word) for word in all_major_answers]
        counter = {k: v for k, v in Counter(all_major_answers).items()} # if v >= 9} # for choosing the major ans
        ans2label = {k: i for i, k in enumerate(counter.keys())}
        label2ans = list(counter.keys())

        for split, annots in zip(
            ["train", "val"], [annotations_train, annotations_val],
        ):
            _annot = annotations[split]

            for q in annots:
                answer_count = {}
                for ad in q['answer']:
                    answer_ = ad["answer"]
                    answer_count[answer_] = answer_count.get(answer_, 0) + 1

                labels = []
                scores = []
                for answer in answer_count:
                    answer_n = normalize_word_nltk(answer)
                    if answer_n not in ans2label:
                        continue

                    labels.append(ans2label[answer_n])
                    score = cls.get_score(answer_count[answer])
                    scores.append(score)

                assert "labels" not in _annot[int(q["image_id"])][q["question_id"]]
                assert "question" in _annot[int(q["image_id"])][q["question_id"]]
                _annot[int(q["image_id"])][q["question_id"]]["labels"] = labels
                _annot[int(q["image_id"])][q["question_id"]]["scores"] = scores

                if (len(_annot[int(q["image_id"])][q["question_id"]]["labels"])) == 0:
                  print(q['image_id'],q['multiple_choice_answer'])


        # ====== TROUBESHOOT POINT ======
        for split,alist in zip(["train", "val"],[annotations_train, annotations_val]):
          print(f"Total annotations in {split} after adding answers: {len(annotations[split])} out of {len(alist)}")

        # - - - filtering - - - (comment out block if not needed)
        # for split in ["train", "val"]:
        #     filtered_annot = dict()
        #     for ik, iv in annotations[split].items():
        #         new_q = dict()
        #         for qk, qv in iv.items():
        #             if len(qv["labels"]) != 0:
        #                 new_q[qk] = qv
        #         if len(new_q) != 0:
        #             filtered_annot[ik] = new_q
        #     annotations[split] = filtered_annot

        # for split in ["train", "val"]:
        #   filtered_annot = dict()
        #   for ik, iv in annotations[split].items():
        #       new_q = dict()
        #       for qk, qv in iv.items():
        #           if len(qv["labels"]) == 0:
        #               print(f"Removing question ID {qk} from image ID {ik} because it has no labels.")
        #               print(annotations[split][ik])
        #           else:
        #               new_q[qk] = qv
        #       if len(new_q) == 0:
        #           print(f"Removing image ID {ik} because it has no questions with labels.")
        #       else:
        #           filtered_annot[ik] = new_q
        #   annotations[split] = filtered_annot

        # # ====== TROUBESHOOT POINT ======
        # for split,alist in zip(["train", "val"],[annotations_train, annotations_val]):
        #   print(f"Total annotations in {split} after filtering: {len(annotations[split])} out of {len(alist)}")

        split2items = {}
        print('\n\nSplitting to items... \n')

        for split in ["train", "val", "test"]:
            print(f"Processing {split}...")
            annot = annotations[split]
            split_name = {
                "train": "train",
                "val": "train",
                "test": "valid"
            }[split]
            paths = list(glob.glob(f"{data_path}/{split_name}/*.jpg"))
            random.shuffle(paths)

            # # ====== TROUBLESHOOT POINT ======
            print(f"Total images in {split}: {len(paths)}")

            annot_paths = [path for path in paths \
                if int(path.split("/")[-1].split("_")[-1][:-4]) in annot]

            # ====== TROUBLESHOOTING ======
            # annot_paths = []
            # for path in paths:
            #     img_id = int(path.split("/")[-1].split("_")[-1][:-4])
            #     if img_id in annot:
            #         annot_paths.append(path)
            #     else:
            #         print(f"Image {path} with ID {img_id} is not in annotations")


        #     # **GEMINI SUGGEST**
        #     # Create a set of image IDs from the annotation data for efficient lookup
        #     annot_image_ids = [int(i) for i in list(set(annot.keys()))]
        #     annot_paths = [path for path in paths if int(path.split("/")[-1].split("_")[-1][:-5]) in annot_image_ids]


            if len(paths) == len(annot_paths):
                print("all images have caption annotations")
            else:
                print("not all images have caption annotations")
            print(len(paths), len(annot_paths), len(annot))

            items = []
            for path in annot_paths:
                iid = int(path.split("/")[-1].split("_")[-1][:-4])
                _annot = annotations[split][iid]
                for qid in _annot:
                    q = _annot[qid]
                    if split in ["train", "val"]:
                        labels = q["labels"]
                        scores = q["scores"]
                    else:
                        labels, scores = [], []

                    items.append({
                        "image_path": os.path.join(split_name, path.split('/')[-1]),
                        "text_segment": q["token_ids"],
                        "labels": labels,
                        "scores": scores,
                        "qid": qid,
                    })

                # **GEMINI: Check if the image ID exists in annotations before accessing it
                # if iid in annotations[split]:
                #     _annot = annotations[split][iid]
                #     for qid in _annot:
                #         q = _annot[qid]
                #         if split in ["train", "val"]:
                #             labels = q["labels"]
                #             scores = q["scores"]
                #         else:
                #             labels, scores = [], []

                #         items.append({
                #             "image_path": os.path.join(split_name, path.split('/')[-1]),
                #             "text_segment": q["token_ids"],
                #             "labels": labels,
                #             "scores": scores,
                #             "qid": qid,
                #         })
                # else:
                #     print(f"Warning: Image ID {iid} not found in annotations for split {split}") # Print a warning if an image ID is missing


            split2items[split] = items
            print(f'len for split2items_{split}: {len(split2items[split])}')

            _write_data_into_jsonl(items=items, jsonl_file=os.path.join(data_path, "vqa.%s.jsonl" % split))

            print(f"Finished processing {split}! \n")

        # Following ViLT, we use 1000 images of the original val set as the final val set
        val_image2items = defaultdict(list)
        for item in split2items["val"]:
            val_image2items[item["image_path"]].append(item)

        print("Contains %d image and %d pairs for val set!" % (len(val_image2items), len(split2items["val"])))

        val_images = list(val_image2items.keys())
        random.shuffle(val_images)
        trainable_val = []
        rest_val = []
        for i, image_id in enumerate(val_images):
            if i < 1000:
                rest_val += val_image2items[image_id]
            else:
                trainable_val += val_image2items[image_id]

        _write_data_into_jsonl(items=trainable_val, jsonl_file=os.path.join(data_path, "vqa.trainable_val.jsonl"))
        _write_data_into_jsonl(items=rest_val, jsonl_file=os.path.join(data_path, "vqa.rest_val.jsonl"))

        with open(os.path.join(data_path, "answer2label.txt"), mode="w", encoding="utf-8") as writer:
            for ans in ans2label:
                to_json = {
                    "answer": ans,
                    "label": ans2label[ans]
                }
                writer.write("%s\n" % json.dumps(to_json))


## make dataset

In [ ]:
# This code creates an index according to the format learned by the BEIT-3 model.
# from datasets import CustomDataset
# from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer("/content/drive/MyDrive/03_AY2425/04_5_DL/_final/beit_models/beit3.spm")

dataset = VQAv2Dataset.make_dataset_index(
    data_path="/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/",
    tokenizer=tokenizer,
    annotation_data_path="/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/beitvqa_json/",
)

Loaded annotations after question processing:
train annotations: 16892 out of 16892
val annotations: 2981 out of 2981
test annotations: 4969 out of 4969
len all_major_answers: 19873
Total annotations in train after adding answers: 16892 out of 16892
Total annotations in val after adding answers: 2981 out of 2981


Splitting to items... 

Processing train...
Total images in train: 19873
not all images have caption annotations
19873 16892 16892
len for split2items_train: 16892
Write /content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/vqa.train.jsonl with 16892 items !
Finished processing train! 

Processing val...
Total images in val: 19873
not all images have caption annotations
19873 2981 2981
len for split2items_val: 2981
Write /content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/vqa.val.jsonl with 2981 items !
Finished processing val! 

Processing test...
Total images in test: 4969
all images have caption annotations
4969 4969 4969
len for split2items_test: 4969
Write /content/drive

# training

## sub1

In [4]:
# %cd /content/drive/MyDrive/_final
# !git clone https://github.com/hmsquare/unilm.git
%cd /content/drive/MyDrive/_final/unilm/beit3
!pip install -r requirements.txt

/content/drive/.shortcut-targets-by-id/1LvXP-6a3tiyiZUKk__pV3AsQXseiHNw9/_final/unilm/beit3


In [ ]:
%cd /content/drive/MyDrive/_final/unilm/beit3
!python run_beit3_finetuning.py \
        --model beit3_base_patch16_384 \
        --input_size 384 \
        --task vqav2 \
        --batch_size 128 \
        --layer_decay 1.0 \
        --lr 2e-5 \
        --update_freq 1 \
        --randaug \
        --epochs 1 \
        --warmup_epochs 0 \
        --drop_path 0.15 \
        --sentencepiece_model /content/drive/MyDrive/_final/beit_models/beit3.spm \
        --finetune /content/drive/MyDrive/_final/beit_models/beit3_base_indomain_patch16_224.pth \
        --data_path /content/drive/MyDrive/_final/data/ \
        --output_dir /content/drive/MyDrive/_final/finetuned_models/epoch1 \
        --log_dir /content/drive/MyDrive/_final/finetuned_models/log \
        --weight_decay 0.01 \
        --seed 42 \
        --save_ckpt_freq 5 \
        --task_head_lr_weight 20 \
        --opt_betas 0.9 0.98 \
        --checkpoint_activations

/content/drive/.shortcut-targets-by-id/1LvXP-6a3tiyiZUKk__pV3AsQXseiHNw9/_final/unilm/beit3
Not using distributed mode
Namespace(model='beit3_base_patch16_384', task='vqav2', input_size=384, drop_path=0.15, checkpoint_activations=True, sentencepiece_model='/content/drive/MyDrive/_final/beit_models/beit3.spm', vocab_size=64010, num_max_bpe_tokens=64, model_ema=False, model_ema_decay=0.9999, model_ema_force_cpu=False, opt='adamw', opt_eps=1e-08, opt_betas=[0.9, 0.98], clip_grad=None, momentum=0.9, weight_decay=0.01, lr=2e-05, layer_decay=1.0, task_head_lr_weight=20.0, warmup_lr=1e-06, min_lr=1e-06, warmup_epochs=0, warmup_steps=-1, batch_size=128, eval_batch_size=None, epochs=1, update_freq=1, save_ckpt_freq=5, randaug=True, train_interpolation='bicubic', finetune='/content/drive/MyDrive/_final/beit_models/beit3_base_indomain_patch16_224.pth', model_key='model|module', model_prefix='', data_path='/content/drive/MyDrive/_final/data/', output_dir='/content/drive/MyDrive/_final/finetuned_mo

In [ ]:
!ls -l | wc -l

## og

In [ ]:
!python run_beit3_finetuning.py \
        --model beit3_base_patch16_384 \
        --input_size 384 \
        --task vqav2 \
        --batch_size 128 \
        --layer_decay 1.0 \
        --lr 2e-5 \
        --update_freq 1 \
        --randaug \
        --epochs 1 \
        --warmup_epochs 0 \
        --drop_path 0.15 \
        --sentencepiece_model /content/drive/MyDrive/03_AY2425/04_5_DL/_final/beit_models/beit3.spm \
        --finetune /content/drive/MyDrive/03_AY2425/04_5_DL/_final/beit_models/beit3_base_indomain_patch16_224.pth \
        --data_path /content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/ \
        --output_dir /content/drive/MyDrive/03_AY2425/04_5_DL/_final/finetuned_models/epoch1_ \
        --log_dir /content/drive/MyDrive/03_AY2425/04_5_DL/_final/finetuned_models/log \
        --weight_decay 0.01 \
        --seed 42 \
        --save_ckpt_freq 5 \
        --task_head_lr_weight 20 \
        --opt_betas 0.9 0.98 \
        --checkpoint_activations

Not using distributed mode
Namespace(model='beit3_base_patch16_384', task='vqav2', input_size=384, drop_path=0.15, checkpoint_activations=True, sentencepiece_model='/content/drive/MyDrive/03_AY2425/04_5_DL/_final/beit_models/beit3.spm', vocab_size=64010, num_max_bpe_tokens=64, model_ema=False, model_ema_decay=0.9999, model_ema_force_cpu=False, opt='adamw', opt_eps=1e-08, opt_betas=[0.9, 0.98], clip_grad=None, momentum=0.9, weight_decay=0.01, lr=2e-05, layer_decay=1.0, task_head_lr_weight=20.0, warmup_lr=1e-06, min_lr=1e-06, warmup_epochs=0, warmup_steps=-1, batch_size=128, eval_batch_size=None, epochs=1, update_freq=1, save_ckpt_freq=5, randaug=True, train_interpolation='bicubic', finetune='/content/drive/MyDrive/03_AY2425/04_5_DL/_final/beit_models/beit3_base_indomain_patch16_224.pth', model_key='model|module', model_prefix='', data_path='/content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/', output_dir='/content/drive/MyDrive/03_AY2425/04_5_DL/_final/finetuned_models/epoch1_', log_d

In [ ]:
!python run_beit3_finetuning.py \
        --model beit3_base_indomain_patch16_224 \
        --input_size 384 \
        --task vqav2 \
        --batch_size 128 \
        --layer_decay 1.0 \
        --lr 2e-5 \
        --update_freq 1 \
        --randaug \
        --epochs 9 \
        --warmup_epochs 0 \
        --drop_path 0.15 \
        --sentencepiece_model /content/drive/MyDrive/03_AY2425/04_5_DL/_final/beit_models/beit3.spm \
        --finetune /content/drive/MyDrive/03_AY2425/04_5_DL/_final/finetuned_models/checkpoint-best.pth \
        --data_path /content/drive/MyDrive/03_AY2425/04_5_DL/_final/data/ \
        --output_dir /content/drive/MyDrive/03_AY2425/04_5_DL/_final/finetuned_models/ \
        --log_dir /content/drive/MyDrive/03_AY2425/04_5_DL/_final/finetuned_models/log \
        --weight_decay 0.01 \
        --seed 42 \
        --save_ckpt_freq 5 \
        --task_head_lr_weight 20 \
        --opt_betas 0.9 0.98 \
        --checkpoint_activations

# inferencing

In [ ]:
!python run_beit3_finetuning.py \
        --model beit3_base_indomain_patch16_224 \
        --input_size 224 \
        --task vqav2 \
        --batch_size 128 \
        --sentencepiece_model /content/drive/MyDrive/Dacon_multimodal/model_file/beit3.spm \
        --finetune /content/drive/MyDrive/Dacon_multimodal/finetuned_model_file/custom_model.pth \
        --data_path /content/ \
        --output_dir /content/drive/MyDrive/Dacon_multimodal/finetuned_model_file/ \
        --eval

# submission

In [ ]:
with open('/content/drive/MyDrive/Dacon_multimodal/finetuned_model_file/submit_vqacustom_test.json', "r") as json_file:
    datas = json.load(json_file)

final_submission = sample_submission.copy()

for data in datas:
    id = 'TEST_' + str(data['question_id'])[1:6]
    final_submission.loc[final_submission['ID'] == id, 'answer'] = data['answer']

final_submission.to_csv('/content/drive/MyDrive/Dacon_multimodal/submission.csv', index=False)

final_submission